In [ ]:
! pip install sqlalchemy
! pip install PyMySQL

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
query_1 = '''
    SELECT customer_id, COUNT(*) AS total_rentals_may
      FROM rentals_may
     GROUP BY customer_id;
'''
data_may = pd.read_sql_query(query_1, engine)
data_may.head()

,customer_id,total_rentals_may
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [4]:
query_2 = '''
    SELECT customer_id, COUNT(*) AS total_rentals_june
      FROM rentals_june
     GROUP BY customer_id;
'''
data_june = pd.read_sql_query(query_2, engine)
data_june.head()

,customer_id,total_rentals_june
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


In [5]:
data_all = data_may.merge(data_june, left_on='customer_id', right_on='customer_id')

In [6]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512 entries, 0 to 511
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   customer_id         512 non-null    int64
 1   total_rentals_may   512 non-null    int64
 2   total_rentals_june  512 non-null    int64
dtypes: int64(3)
memory usage: 16.0 KB


In [7]:
data_all.sample(10)

,customer_id,total_rentals_may,total_rentals_june
364,588,2,5
468,520,1,3
359,463,1,4
7,239,6,5
357,55,3,5
39,413,2,2
399,338,1,7
20,388,2,5
501,481,1,4
97,310,1,6


In [8]:
# check if there are NaN values
data_all.isnull().sum()

customer_id           0
total_rentals_may     0
total_rentals_june    0
dtype: int64

In [9]:
def has_june_lending_increased(c_id):
    index = data_all.index[data_all.customer_id == c_id].tolist()[0]
    return data_all.at[index,'total_rentals_june'] > data_all.at[index,'total_rentals_may']

In [10]:
# some tests 
assert has_june_lending_increased(361) == True
assert has_june_lending_increased(9) == False
assert has_june_lending_increased(432) == False